In [21]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%debug

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
> /usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py(1022)fit_transform()
   1020         if isinstance(raw_documents, six.string_types):
   1021             raise ValueError(
-> 1022                 "Iterable over raw text documents expected, "
   1023                 "string object received.")
   1024 

ipdb> q


In [2]:
import sys
sys.path.insert(0, "../../code-previous")

In [3]:
import pandas as pd
import numpy as np
import networkx as nx
import sklearn.feature_extraction as skfeatures
import time
import os

import operator

import string
import swifter
import nltk
stopwords = nltk.corpus.stopwords.words('english')


import plotly
import plotly.plotly as py
from plotly import tools

import plotly.graph_objs as go
plotly.tools.set_credentials_file(username='sreejith_s', api_key='rcet0rTqFCc3WeOzWTjv')



from sklearn.feature_extraction.text import CountVectorizer


from unicodedata import category
from tqdm import tqdm
tqdm.pandas(desc="Progress")

import seaborn as sns
sns.set_style('white')
import matplotlib.pylab as plt

/usr/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [4]:
data_path = "/mnt/disks/vault/analysis-data/interdisciplinary-novelty-analysis/all_discipline_data.pql"
data = pd.read_pickle(data_path)

In [5]:
data50 = data[data.PubYear == 1950]

In [6]:
data50_group = data50[["Label","title_without_stopwords"]].groupby(data50.Label)["title_without_stopwords"].apply(list)

In [7]:
data50_group = pd.DataFrame(data50_group)

In [8]:
data50_group[:5]

,title_without_stopwords
Label,
1ST YEAR,[hyperbrachycephaly influenced cultural condit...
9-YEAR-OLD WHITE MALES,[distribution subcutaneous]
ABILITIES,"[construction validation educational tests, in..."
ACCEPTANCE,[counseling methods therapy]
ACID,[connective tissue fine structure some methods...


In [9]:
data50_group["title_words"] = data50_group["title_without_stopwords"].apply(lambda x: " ".join(x))

In [10]:
data50_group[:10]

,title_without_stopwords,title_words
Label,,
1ST YEAR,[hyperbrachycephaly influenced cultural condit...,hyperbrachycephaly influenced cultural conditi...
9-YEAR-OLD WHITE MALES,[distribution subcutaneous],distribution subcutaneous
ABILITIES,"[construction validation educational tests, in...",construction validation educational tests inte...
ACCEPTANCE,[counseling methods therapy],counseling methods therapy
ACID,[connective tissue fine structure some methods...,connective tissue fine structure some methods ...
ADIPOSE-TISSUE,[distribution subcutaneous],distribution subcutaneous
ADOLESCENT GIRLS,[card versus booklet forms mmpi],card versus booklet forms mmpi
ADRENAL-CORTICAL RESPONSES,[abnormalities behavior],abnormalities behavior
ADULTS,[rorschach human other movement responses inre...,rorschach human other movement responses inrel...


In [11]:
vectorizer = CountVectorizer()
vectorized_words = vectorizer.fit_transform(data50_group.title_words)

In [12]:
vocab = vectorizer.vocabulary_
rev_vocab = {index:word for word,index in vocab.items()}

In [15]:
all_discipline_word_frequencies = vectorized_words.sum(axis=0).reshape(-1,).tolist()[0]
word_index_with_freq = {index:freq for index,freq in enumerate(all_discipline_word_frequencies)}
word_with_freq = {rev_vocab[index]:np.log(freq + 1) for index, freq in word_index_with_freq.items()}

In [38]:
# document frequency

vectorizers = []
discipline_word_freq = []

for i in range(len(data50_group)):
    words_for_discipline = data50_group.iloc[i]["title_words"]
    
    tmp_vectorizer = CountVectorizer()
    tmp_vectorized_words = tmp_vectorizer.fit_transform(words_for_discipline.split(" "))
    
    vectorizers.append(tmp_vectorizer)
    
    vocab_ = tmp_vectorizer.vocabulary_
    rev_vocab = {index:word for word,index in vocab_.items()}
    
    discipline_word_frequencies = tmp_vectorized_words.sum(axis=0).reshape(-1,).tolist()[0]
    disc_word_index_with_freq = {index:freq for index,freq in enumerate(discipline_word_frequencies)}
    
    disc_word_with_freq = {rev_vocab[index]:np.log(freq + 1) for index, freq in disc_word_index_with_freq.items()}
    
    discipline_word_freq.append(disc_word_with_freq)

In [39]:
# data50_group["title_words"].apply(lambda x: len(x.split(" ")))[:5]

discipline_word_freq[10]

{'among': 0.6931471805599453,
 'apparent': 0.6931471805599453,
 'associates': 0.6931471805599453,
 'cases': 0.6931471805599453,
 'decline': 1.0986122886681098,
 'development': 0.6931471805599453,
 'growth': 0.6931471805599453,
 'household': 0.6931471805599453,
 'infection': 0.6931471805599453,
 'positive': 0.6931471805599453,
 'sputum': 0.6931471805599453,
 'tuberculosis': 0.6931471805599453,
 'tuberculous': 0.6931471805599453}